# Spatial Joins Exercises

Here\'s a reminder of some of the functions we have seen. Hint: they
should be useful for the exercises!

-   `sum(expression)`: aggregate to
    return a sum for a set of records
-   `count(expression)`: aggregate to
    return the size of a set of records
-   `ST_Area(geometry)` returns the
    area of the polygons
-   `ST_AsText(geometry)` returns WKT `text`
-   `ST_Contains(geometry A, geometry B)` returns the true if geometry A contains geometry B
-   `ST_Distance(geometry A, geometry B)` returns the minimum distance between geometry A and
    geometry B
-   `ST_DWithin(geometry A, geometry B, radius)` returns the true if geometry A is radius distance or less from geometry B
-   `ST_GeomFromText(text)` returns `geometry`
-   `ST_Intersects(geometry A, geometry B)` returns the true if geometry A intersects geometry B
-   `ST_Length(linestring)` returns the length of the linestring
-   `ST_Touches(geometry A, geometry B)` returns the true if the boundary of geometry A touches geometry B
-   `ST_Within(geometry A, geometry B)` returns the true if geometry A is within geometry B


Uncomment and run the following cell to install the required packages.


In [1]:
%pip install duckdb leafmap lonboard
import duckdb
import leafmap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 884.2/884.2 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.7/213.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.5/765.5 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.2/194.2 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 

Download the [nyc_data.zip](https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip) dataset using leafmap. The zip file contains the following datasets. Create a new DuckDB database and import the datasets into the database. Each dataset should be imported into a separate table.

- nyc_census_blocks
- nyc_homicides
- nyc_neighborhoods
- nyc_streets
- nyc_subway_stations

In [2]:
url = "https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip"
leafmap.download_file(url, unzip=True)

Downloading...
From: https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip
To: /content/nyc_data.zip
100%|██████████| 8.73M/8.73M [00:00<00:00, 57.1MB/s]


Extracting files...


'/content/nyc_data.zip'

In [3]:
con = duckdb.connect()

In [4]:
con.install_extension("spatial")
con.load_extension("spatial")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [17]:
# 使用 ST_Read 函数创建表并导入数据
con.sql("CREATE TABLE nyc_census_blocks AS SELECT * FROM ST_Read('nyc_census_blocks.shp')")

CatalogException: Catalog Error: Table with name "nyc_census_blocks" already exists!

In [18]:
con.sql("CREATE TABLE nyc_homicides AS SELECT * FROM ST_Read('nyc_homicides.shp')")
con.sql("CREATE TABLE nyc_neighborhoods AS SELECT * FROM ST_Read('nyc_neighborhoods.shp')")
con.sql("CREATE TABLE nyc_streets AS SELECT * FROM ST_Read('nyc_streets.shp')")

CatalogException: Catalog Error: Table with name "nyc_homicides" already exists!

In [21]:
con.sql("CREATE TABLE nyc_subway_stations AS SELECT * FROM ST_Read('nyc_subway_stations.shp')")

In [52]:
df = con.sql("SELECT * FROM nyc_subway_stations").df()
df.head()

,OBJECTID,ID,NAME,ALT_NAME,CROSS_ST,LONG_NAME,LABEL,BOROUGH,NGHBHD,ROUTES,TRANSFERS,COLOR,EXPRESS,CLOSED,geom
0,1.0,376.0,Cortlandt St,None,Church St,"Cortlandt St (R,W) Manhattan","Cortlandt St (R,W)",Manhattan,None,"R,W","R,W",YELLOW,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,2.0,2.0,Rector St,None,None,Rector St (1) Manhattan,Rector St (1),Manhattan,None,1,1,RED,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,3.0,1.0,South Ferry,None,None,South Ferry (1) Manhattan,South Ferry (1),Manhattan,None,1,1,RED,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,4.0,125.0,138th St,Grand Concourse,Grand Concourse,"138th St / Grand Concourse (4,5) Bronx","138th St / Grand Concourse (4,5)",Bronx,None,"4,5","4,5",GREEN,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,5.0,126.0,149th St,Grand Concourse,Grand Concourse,149th St / Grand Concourse (4) Bronx,149th St / Grand Concourse (4),Bronx,None,4,"2,4,5",GREEN,express,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


In [53]:
con.sql("SELECT * FROM nyc_neighborhoods").df().head()


,BORONAME,NAME,geom
0,Brooklyn,Bensonhurst,"[2, 4, 0, 0, 0, 0, 0, 0, 54, 71, 14, 73, 198, ..."
1,Manhattan,East Village,"[2, 4, 0, 0, 0, 0, 0, 0, 35, 215, 14, 73, 139,..."
2,Manhattan,West Village,"[2, 4, 0, 0, 0, 0, 0, 0, 161, 95, 14, 73, 212,..."
3,The Bronx,Throggs Neck,"[2, 4, 0, 0, 0, 0, 0, 0, 128, 232, 17, 73, 174..."
4,The Bronx,Wakefield-Williamsbridge,"[2, 4, 0, 0, 0, 0, 0, 0, 83, 85, 17, 73, 17, 2..."


In [54]:
con.sql("SELECT * FROM nyc_homicides").df().head()

,INCIDENT_D,BORONAME,NUM_VICTIM,PRIMARY_MO,ID,WEAPON,LIGHT_DARK,YEAR,geom
0,2008-01-01,Brooklyn,1,None,7,gun,D,2008,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,2008-01-04,Manhattan,1,None,14,gun,D,2008,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,2008-01-05,Queens,1,None,15,gun,D,2008,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,2008-01-04,Queens,1,None,16,knife,D,2008,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,2008-01-05,Queens,1,None,18,gun,D,2008,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


In [55]:
con.sql("SELECT * FROM nyc_census_blocks").df().head()

,BLKID,POPN_TOTAL,POPN_WHITE,POPN_BLACK,POPN_NATIV,POPN_ASIAN,POPN_OTHER,BORONAME,geom
0,360850009001000,97,51,32,1,5,8,Staten Island,"[2, 4, 0, 0, 0, 0, 0, 0, 55, 3, 13, 73, 151, 8..."
1,360850020011000,66,52,2,0,7,5,Staten Island,"[2, 4, 0, 0, 0, 0, 0, 0, 178, 58, 13, 73, 72, ..."
2,360850040001000,62,14,18,2,25,3,Staten Island,"[2, 4, 0, 0, 0, 0, 0, 0, 82, 227, 12, 73, 55, ..."
3,360850074001000,137,92,12,0,13,20,Staten Island,"[2, 4, 0, 0, 0, 0, 0, 0, 204, 85, 13, 73, 103,..."
4,360850096011000,289,230,0,0,32,27,Staten Island,"[2, 4, 0, 0, 0, 0, 0, 0, 107, 247, 12, 73, 7, ..."


In [56]:
con.sql("SELECT * FROM nyc_streets").df().head()

,ID,NAME,ONEWAY,TYPE,geom
0,1,Shore Pky S,None,residential,"[1, 4, 0, 0, 0, 0, 0, 0, 23, 66, 15, 73, 202, ..."
1,2,None,None,footway,"[1, 4, 0, 0, 0, 0, 0, 0, 80, 57, 15, 73, 35, 1..."
2,3,Avenue O,None,residential,"[1, 4, 0, 0, 0, 0, 0, 0, 228, 63, 15, 73, 219,..."
3,4,Walsh Ct,None,residential,"[1, 4, 0, 0, 0, 0, 0, 0, 139, 62, 15, 73, 102,..."
4,5,None,None,motorway_link,"[1, 4, 0, 0, 0, 0, 0, 0, 176, 53, 15, 73, 16, ..."


1. **What subway station is in \'Little Italy\'? What subway route is it on?**

In [61]:
con.sql("""SELECT nss.*, nn.BORONAME
FROM  nyc_subway_stations nss
LEFT JOIN nyc_neighborhoods nn
ON nss.BOROUGH = nn.BORONAME""")



┌──────────┬────────┬──────────────┬────────────────┬────────────────────┬───────────────────────────────────────────┬─────────────────────────────────┬───────────┬─────────┬─────────┬───────────────┬──────────────┬─────────┬─────────┬──────────────────────────────────────────────┬───────────┐
│ OBJECTID │   ID   │     NAME     │    ALT_NAME    │      CROSS_ST      │                 LONG_NAME                 │              LABEL              │  BOROUGH  │ NGHBHD  │ ROUTES  │   TRANSFERS   │    COLOR     │ EXPRESS │ CLOSED  │                     geom                     │ BORONAME  │
│  double  │ double │   varchar    │    varchar     │      varchar       │                  varchar                  │             varchar             │  varchar  │ varchar │ varchar │    varchar    │   varchar    │ varchar │ varchar │                   geometry                   │  varchar  │
├──────────┼────────┼──────────────┼────────────────┼────────────────────┼─────────────────────────────────────────

In [62]:
con.sql("""
SELECT *
FROM nyc_subway_stations
WHERE LABEL LIKE '%Little Italy%'
""")


┌──────────┬────────┬─────────┬──────────┬──────────┬───────────┬─────────┬─────────┬─────────┬─────────┬───────────┬─────────┬─────────┬─────────┬──────────┐
│ OBJECTID │   ID   │  NAME   │ ALT_NAME │ CROSS_ST │ LONG_NAME │  LABEL  │ BOROUGH │ NGHBHD  │ ROUTES  │ TRANSFERS │  COLOR  │ EXPRESS │ CLOSED  │   geom   │
│  double  │ double │ varchar │ varchar  │ varchar  │  varchar  │ varchar │ varchar │ varchar │ varchar │  varchar  │ varchar │ varchar │ varchar │ geometry │
├──────────┴────────┴─────────┴──────────┴──────────┴───────────┴─────────┴─────────┴─────────┴─────────┴───────────┴─────────┴─────────┴─────────┴──────────┤
│                                                                           0 rows                                                                           │
└────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

2. **What are all the neighborhoods served by the 6-train?** (Hint: The `routes` column in the `nyc_subway_stations` table has values like \'B,D,6,V\' and \'C,6\')


In [64]:
# prompt: What are all the neighborhoods served by the 6-train? (Hint: The routes column in the nyc_subway_stations table has values like 'B,D,6,V' and 'C,6')

con.sql("""
SELECT DISTINCT n.name
FROM nyc_subway_stations AS s
JOIN nyc_neighborhoods AS n ON ST_Within(s.geom, n.geom)
WHERE s.routes LIKE '%6%'
""")


┌────────────────────┐
│        NAME        │
│      varchar       │
├────────────────────┤
│ Chinatown          │
│ Greenwich Village  │
│ Murray Hill        │
│ Midtown            │
│ Upper East Side    │
│ Yorkville          │
│ Hunts Point        │
│ South Bronx        │
│ Soundview          │
│ Parkchester        │
│ Financial District │
│ Little Italy       │
│ Gramercy           │
│ East Harlem        │
│ Mott Haven         │
├────────────────────┤
│      15 rows       │
└────────────────────┘

3. **After 9/11, the \'Battery Park\' neighborhood was off limits for several days. How many people had to be evacuated?**

In [66]:
# prompt: After 9/11, the 'Battery Park' neighborhood was off limits for several days. How many people had to be evacuated?

con.sql("""
SELECT SUM(POPN_TOTAL)
FROM nyc_census_blocks AS c
JOIN nyc_neighborhoods AS n ON ST_Intersects(c.geom, n.geom)
WHERE n.name = 'Battery Park'
""")


┌─────────────────┐
│ sum(POPN_TOTAL) │
│     int128      │
├─────────────────┤
│           17153 │
└─────────────────┘

4. **What neighborhood has the highest population density (persons/km2)?**


In [69]:
# prompt: What neighborhood has the highest population density (persons/km2)?

con.sql("""
SELECT BORONAME
FROM nyc_census_blocks
ORDER BY POPN_TOTAL / ST_Area(geom) DESC
LIMIT 1
""")


┌───────────┐
│ BORONAME  │
│  varchar  │
├───────────┤
│ Manhattan │
└───────────┘

When you're finished, you can check your answers [here](https://postgis.net/workshops/postgis-intro/joins_exercises.html).

# Ship-to-Ship Transfer Detection

Now for a less structured exercise. We're going to look at ship-to-ship transfers. The idea is that two ships meet up in the middle of the ocean, and one ship transfers cargo to the other. This is a common way to avoid sanctions, and is often used to transfer oil from sanctioned countries to other countries. We're going to look at a few different ways to detect these transfers using AIS data.

In [70]:
%pip install duckdb duckdb-engine jupysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.2 MB/s eta 0:00:00


In [71]:
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

In [72]:
%%sql
INSTALL httpfs;
LOAD httpfs;
INSTALL spatial;
LOAD spatial;

,Success


## Step 1

Create a spatial database using the following AIS data:

https://storage.googleapis.com/qm2/casa0025_ships.csv

Each row in this dataset is an AIS 'ping' indicating the position of a ship at a particular date/time, alongside vessel-level characteristics.

It contains the following columns:
* `vesselid`: A unique numerical identifier for each ship, like a license plate
* `vessel_name`: The ship's name
* `vsl_descr`: The ship's type
* `dwt`: The ship's Deadweight Tonnage (how many tons it can carry)
* `v_length`: The ship's length in meters
* `draught`: How many meters deep the ship is draughting (how low it sits in the water). Effectively indicates how much cargo the ship is carrying
* `sog`: Speed over Ground (in knots)
* `date`: A timestamp for the AIS signal
* `lat`: The latitude of the AIS signal (EPSG:4326)
* `lon`: The longitude of the AIS signal (EPSG:4326)

Create a table called 'ais' where each row is a different AIS ping, with no superfluous information. Construct a geometry column.

Create a second table called 'vinfo' which contains vessel-level information with no superfluous information.

You can set a spatial index on each of these tables as follows:

`CREATE INDEX index_name ON table_name USING RTREE(geom);`

In [73]:
# prompt: Create a spatial database using the following AIS data:
# 使用以下 AIS 数据创建空间数据库：
# https://storage.googleapis.com/qm2/casa0025_ships.csv


# Create a DuckDB connection
con = duckdb.connect()

# Install and load spatial extension
con.install_extension("spatial")
con.load_extension("spatial")

# Read the AIS data from the URL
ais_df = pd.read_csv("https://storage.googleapis.com/qm2/casa0025_ships.csv")

# Create the 'ais' table
con.execute("""
CREATE TABLE ais AS
SELECT
    vesselid,
    date,
    sog,
    draught,
    ST_Point(lon, lat) AS geom
FROM ais_df
""")

# Create a spatial index on the 'ais' table
con.execute("CREATE INDEX ais_geom_idx ON ais USING RTREE(geom)")


# Create the 'vinfo' table
con.execute("""
CREATE TABLE vinfo AS
SELECT DISTINCT
    vesselid,
    vessel_name,
    vsl_descr,
    dwt,
    v_length
FROM ais_df
""")

# Display the first few rows of each table to verify
print("AIS table:")
print(con.execute("SELECT * FROM ais LIMIT 5").df())

print("\nVINFO table:")
print(con.execute("SELECT * FROM vinfo LIMIT 5").df())


AIS table:
   vesselid                 date  sog  draught  \
0    350053  2022-07-25 02:53:29  5.2      3.5   
1    350053  2022-07-25 03:09:37  0.7      3.5   
2    350053  2022-07-25 03:13:58  0.7      3.5   
3    350053  2022-07-25 04:16:06  0.1      3.5   
4    350053  2022-07-25 05:20:17  0.0      3.5   

                                                geom  
0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...  
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...  
2  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...  
3  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...  
4  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...  

VINFO table:
   vesselid vessel_name                                 vsl_descr       dwt  \
0    142540    A Plus-1     general cargo with container capacity    4742.0   
1    213151    Absheron     general cargo with container capacity    3344.0   
2    330665     Adafera                          crude oil tanker  105215.0   
3    269668     Adam- A     

## Step 2

Use a spatial join to identify ship-to-ship transfers in this dataset.
Two ships are considered to be conducting a ship to ship transfer IF:

* They are within 500 meters of each other
* For more than two hours
* And their speed is lower than 1 knot

Some things to consider: make sure you're not joining ships with themselves. Try working with subsets of the data first while you try different things out.

In [74]:
# prompt: Use a spatial join to identify ship-to-ship transfers in this dataset. Two ships are considered to be conducting a ship to ship transfer IF:
# 使用空间连接来识别此数据集中的船到船转移。如果满足以下条件，则两艘船被视为正在进行船对船转运：
# They are within 500 meters of each other
# 它们彼此相距不到 500 米
# For more than two hours
# 两个多小时
# And their speed is lower than 1 knot
# 而且它们的速度低于 1 节
# Some things to consider: make sure you're not joining ships with themselves. Try working with subsets of the data first while you try different things out.
# 需要考虑的一些事项：确保您没有与自己一起加入船只。先尝试使用数据的子集，然后再尝试不同的作。

%%sql
SELECT a1.vesselid AS vessel1, a2.vesselid AS vessel2
FROM ais AS a1
INNER JOIN ais AS a2
  ON ST_DWithin(a1.geom, a2.geom, 0.5) -- 500 meters
WHERE a1.vesselid != a2.vesselid
  AND a1.sog < 1 AND a2.sog < 1
  AND ABS(strftime('%J', a1.date) - strftime('%J', a2.date)) * 24 * 60 * 60 +
      ABS(strftime('%S', a1.date) - strftime('%S', a2.date)) > 2 * 60 * 60; -- More than 2 hours


RuntimeError: If using snippets, you may pass the --with argument explicitly.
For more details please refer: https://jupysql.ploomber.io/en/latest/compose.html#with-argument


Original error message from DB driver:
(duckdb.duckdb.CatalogException) Catalog Error: Table with name ais does not exist!
Did you mean "pg_namespace"?
LINE 2: FROM ais AS a1
             ^
[SQL: SELECT a1.vesselid AS vessel1, a2.vesselid AS vessel2
FROM ais AS a1
INNER JOIN ais AS a2
  ON ST_DWithin(a1.geom, a2.geom, 0.5)
WHERE a1.vesselid != a2.vesselid
  AND a1.sog < 1 AND a2.sog < 1
  AND ABS(strftime('%J', a1.date) - strftime('%J', a2.date)) * 24 * 60 * 60 +
      ABS(strftime('%S', a1.date) - strftime('%S', a2.date)) > 2 * 60 * 60;]
(Background on this error at: https://sqlalche.me/e/20/f405)

If you need help solving this issue, send us a message: https://ploomber.io/community
